In [1]:
import numpy as np

In [2]:
def generate_sequence(lower, upper, n, q):
    seq = np.random.randint(lower, upper, size=(q, np.random.choice(n)))
    return seq
generate_sequence(1, 15, [3,6,9], 2)

array([[ 3, 14, 13],
       [ 2,  4, 10]])

In [3]:
import tensorflow as tf

/home/varshavskiisd/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
PAD = 0
EOS = 3
max_len = 10
loss_track = list()
score = list()
train_size = None
test_size = None

In [29]:
class SeqToSeq:
    vocab_size = 15
    
    def __init__(self, hidden_units, optimizer=tf.train.AdamOptimizer()):
        tf.reset_default_graph()
        
        # Placeholders and variables used in graph
        self.in_encoder = tf.placeholder(shape=(None, None), dtype=tf.int32)
        self.in_decoder = tf.placeholder(shape=(None, None), dtype=tf.int32)
        self.out_decoder = tf.placeholder(shape=(None, None), dtype=tf.int32)
        
        input_embedding_size = 20
        embeddings = tf.Variable(tf.random_uniform([self.vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)
        encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, self.in_encoder)
        decoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, self.in_decoder)
        
        # encoder
        tmp1 = [
            hidden_units[0],
            encoder_inputs_embedded
        ]
        encoder_outputs, encoder_final_state = self.encoder(tmp1)
        
        # decoder
        tmp2 = [
            hidden_units[1],
            decoder_inputs_embedded,
            encoder_final_state
        ]
        decoder_outputs, decoder_final_state = self.decoder(tmp2)
        
        decoder_logits = tf.contrib.layers.linear(decoder_outputs, self.vocab_size)
        self.decoder_prediction = tf.argmax(decoder_logits, 2)
        
        # Initialisation of loss funstion and optimisation method with constraits for weights changing
        self._loss = self.cost([self.out_decoder, decoder_logits])
        self._optimizer = optimizer.minimize(self._loss)
        
        
    def encoder(self, inner):
        encoder_cell = tf.contrib.rnn.LSTMCell(inner[0])
        encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(encoder_cell, inner[1],
                                                                 dtype=tf.float32,
                                                                 time_major=True)
        
        return encoder_outputs, encoder_final_state

    def decoder(self, inner):
        decoder_cell = tf.contrib.rnn.LSTMCell(inner[0])
        decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(decoder_cell, inner[1],
                                                                 initial_state=inner[2],
                                                                 dtype=tf.float32, time_major=True,
                                                                 scope="plain_decoder")
        return decoder_outputs, decoder_final_state
    
    def cost(self, inner):
        stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.one_hot(inner[0], depth=self.vocab_size, dtype=tf.float32),
            logits=inner[1])
        
        loss = tf.reduce_mean(stepwise_cross_entropy)
        return loss
    
    @property
    def loss(self):
        return self._loss
    
    @property
    def optimze(self):
        return self._optimizer
    
class Process:
    
    def __init__(self, data, params):
        self._epochs, self._batch_size = params['epochs'], params['batch_size']
        init = tf.global_variables_initializer()

        self._sess = tf.Session()
        self._sess.run(init)

        self._data, last, self._iterations = list(), 0, int(data.shape[0] / self._batch_size)
        for i in range(self._iterations-1):
            if i != 0:
                self._data.append(data[last:i * self._batch_size])
                last = i * self._batch_size
    
    def run(self, n, graph):
        PAD, EOS = 0, 1
        for e in range(self._epochs):
            for i in range(self._iterations-1):
                batch = self._data[np.random.randint(len(self._data)-1)]
                feed = {
                    graph.in_encoder : np.array([[PAD] + [j for j in i] for i in batch]),
                    graph.in_decoder : np.array([[EOS] + [j for j in i] for i in batch]),
                    graph.out_decoder : np.array([[j for j in i] + [EOS] for i in batch])
                }
                _,l = self._sess.run([graph.optimze, graph.loss], feed_dict=feed)
            if e % n == 0:
                print('Epoch:', e, 'Loss:', l)
                
        return self._sess

In [30]:
data = generate_sequence(2, 15, [3,4], 100)
base = len(data[0])
for idx, i in enumerate(data):
    r = base - np.random.randint(2)
    data[idx] = [PAD] * (base-r) + [j for jdx, j in enumerate(i) if jdx < r]
data[:2]

array([[ 6, 14,  7,  2],
       [11,  4,  9, 13]])

In [31]:
tf.reset_default_graph()
seq2seq = SeqToSeq(hidden_units=[10, 10])
process = Process(data, { 'batch_size' : 20, 'epochs' : 1000})
process.run(100, seq2seq)

Epoch: 0 Loss: 2.6635756
Epoch: 100 Loss: 1.6237664
Epoch: 200 Loss: 1.1696355
Epoch: 300 Loss: 0.7944676
Epoch: 400 Loss: 0.7023334
Epoch: 500 Loss: 0.53384244
Epoch: 600 Loss: 0.54649156
Epoch: 700 Loss: 0.42520565
Epoch: 800 Loss: 0.39173388
Epoch: 900 Loss: 0.34690636


In [75]:
asd, last, iterations = list(), 0, int(data.shape[0] / 20)
for i in range(iterations):
    if i != 0:
        asd.append(data[last:i * 20])
        last = i * 2
asd[0].shape

(20, 4)

In [80]:
seq2seq = SeqToSeq(hidden_units=[10, 10])
sess = tf.Session()
sess.run(tf.initialize_all_variables())

for epoch in range(1001):
    tmp = iter(asd)
    for i in range(iterations-1):
        batch = next(tmp)#asd[np.random.randint(5-1)]
        feed = {
            seq2seq.in_encoder : np.array([[PAD] + [j for j in i] for i in batch]),
            seq2seq.in_decoder : np.array([[EOS] + [j for j in i] for i in batch]),
            seq2seq.out_decoder : np.array([[j for j in i] + [EOS] for i in batch])
        }
        _,l = sess.run([seq2seq.optimze, seq2seq.loss], feed_dict=feed)
    if epoch % 500 == 0:
        print('Epoch : ' + str(epoch),' Loss : ' + str(l))
        feed = {
                seq2seq.in_encoder : np.array([[PAD] +[j for j in i] for i in asd[0]]),
                seq2seq.in_decoder : np.array([[EOS] + [j for j in i] for i in asd[0]])
        }
        pred = sess.run(seq2seq.decoder_prediction, feed_dict=feed)
        for k, (inp, pred) in enumerate(zip(feed[seq2seq.in_encoder], pred)):
            print('  sample {}:'.format(k + 1))
            print('    input     > {}'.format(inp))
            print('    predicted > {}'.format(pred))
            if k > 2:
                break

Epoch : 0  Loss : 2.752517
  sample 1:
    input     > [ 0  0  7  8 10]
    predicted > [13  0  4  3  3]
  sample 2:
    input     > [ 0 13  5 13 13]
    predicted > [13  9  4  1  9]
  sample 3:
    input     > [ 0  8 13  4  9]
    predicted > [13  0  4  5  9]
  sample 4:
    input     > [0 0 5 3 9]
    predicted > [13  0  4  1  9]
Epoch : 500  Loss : 1.3065455
  sample 1:
    input     > [ 0  0  7  8 10]
    predicted > [ 0  5  8 13  3]
  sample 2:
    input     > [ 0 13  5 13 13]
    predicted > [ 0  5 13 13  3]
  sample 3:
    input     > [ 0  8 13  4  9]
    predicted > [ 0 13  4  9  3]
  sample 4:
    input     > [0 0 5 3 9]
    predicted > [0 5 3 9 3]
Epoch : 1000  Loss : 1.00697
  sample 1:
    input     > [ 0  0  7  8 10]
    predicted > [ 0  7  8 10  3]
  sample 2:
    input     > [ 0 13  5 13 13]
    predicted > [13  5 13 13  3]
  sample 3:
    input     > [ 0  8 13  4  9]
    predicted > [ 8 13  4  9  3]
  sample 4:
    input     > [0 0 5 3 9]
    predicted > [0 5 3 9 3]
